# Signal files
First let's look at the available signal files

In [ ]:
%%bash 
export XrdSecGSISRVNAMES=cmseos.fnal.gov
xrdfs root://cmseos.fnal.gov/ ls /eos/uscms/store/group/lpcjme/noreplica/NanoHRT/Trees/Apr08/forMCComparions/reweighted/

# Open signal files and inspect tree
Let's open the one signal file and one background file and print the contents of the tree

In [ ]:
import ROOT as rt

filename_hh = 'root://cmseos.fnal.gov//eos/uscms/store/group/lpcjme/noreplica/NanoHRT/Trees/Apr08/forMCComparions/reweighted/BulkGravTohh_tree_noc.root'
filename_qcd = 'root://cmseos.fnal.gov//eos/uscms/store/group/lpcjme/noreplica/NanoHRT/Trees/Apr08/forMCComparions/reweighted/qcd-pythia_tree_SLIM.root'

file_hh = rt.TFile.Open(filename_hh)
file_qcd = rt.TFile.Open(filename_qcd)

tree_hh = file_hh.Get('Friends')
tree_qcd = file_qcd.Get('Friends')
tree_hh.Print()

# Plot tagger distributions
Let's plot each of the different discriminatig variables distributions 

In [ ]:
%jsroot on

tagger = 'ak8_sdmass'
#tagger = 'ak8_tau21'
#tagger = 'ak8_ecfN2'
#tagger = 'ak8_doubleb'
#tagger = 'ak8_nn_HbbvsQCD'
#tagger = 'ak8_decorr_nn_ZHbbvsQCD'
#tagger = 'ak8_bestH'

cuts = 'gen_pt > 1000 && gen_pt < 1500 && abs(gen_eta) < 2.4 && ak8_sdmass > 40 && ak8_sdmass < 200'

hist_hh = rt.TH1D('hist_hh','hist_hh',85,30,200)
hist_qcd = rt.TH1D('hist_qcd','hist_qcd',85,30,200)
tree_hh.Project('hist_hh',tagger,'rewgt*(%s)'%cuts)
tree_qcd.Project('hist_qcd',tagger,'genweight*(%s)'%cuts)

c = rt.TCanvas('c','c',500,400)
hist_hh.Scale(1/hist_hh.Integral())
hist_qcd.Scale(1/hist_qcd.Integral())
hist_qcd.SetLineColor(rt.kBlue)
hist_hh.SetLineColor(rt.kRed)
if hist_qcd.GetMaximum() > hist_hh.GetMaximum():
    hist_qcd.Draw('h')
    hist_hh.Draw('hsame')
else:
    hist_hh.Draw('h')
    hist_qcd.Draw('hsame')
c.Draw()

# checkpoint: 
plot the output of the ML-based algorithms: e.g. for the Higgs case for DoubleB, DeepAK8 and DeepAK8-MD

In [ ]:
## place holder for the checkpoint

# Convert tree to dataframe with `uproot`
Open the file with `uproot` and convert the tree to a `pandas` dataframe.

In [ ]:
## this step may take some time 

import uproot
#!pip install lz4 --user
#!pip install xxhash --user

uptree_hh = uproot.open(filename_hh)["Friends"]
uptree_qcd = uproot.open(filename_qcd)["Friends"]

branches = ['ak8_nn_HbbvsQCD',
            'ak8_decorr_nn_ZHbbvsQCD',
            'ak8_doubleb',
            'ak8_tau21',
            'ak8_ecfN2',
            'ak8_bestH',
            'ak8_sdmass',
            'ak8_pt',
            'ak8_eta',
            'gen_pt',
            'gen_eta']


print("get hh")
df_hh = uptree_hh.pandas.df(branches=branches+['rewgt'])
#print(df_hh)

print("get QCD")
df_qcd = uptree_qcd.pandas.df(branches=branches+['genweight'])
#print(df_qcd)

# Apply cuts and merge dataframes
Let's apply the same fiducial cuts and merge the signal and background dataframes

In [ ]:
import pandas
import numpy as np

# mask higgs outside of fiducial cuts
mask_hh = (df_hh['gen_pt'] > 1000) & (df_hh['gen_pt'] < 1500) & (np.abs(df_hh['gen_eta']) < 2.4) & (df_hh['ak8_sdmass'] > 40) & (df_hh['ak8_sdmass'] < 200)
df_hh = df_hh[mask_hh]
df_hh['sample_weight'] = df_hh['rewgt']
df_hh['label'] = np.ones((len(df_hh)),dtype=int)

# mask qcd outside of fiducial cuts
mask_qcd = (df_qcd['gen_pt'] > 1000) & (df_qcd['gen_pt'] < 1500) & (np.abs(df_qcd['gen_eta']) < 2.4) & (df_qcd['ak8_sdmass'] > 40) & (df_qcd['ak8_sdmass'] < 200)
df_qcd = df_qcd[mask_qcd]
df_qcd['sample_weight'] = df_qcd['genweight']
df_qcd['label'] = np.zeros((len(df_qcd)),dtype=int)

# concatenate signal and background
df = pandas.concat([df_hh,df_qcd])

# Plot ROCs
Plot the ROC for one of the tagging variables

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
%matplotlib inline
fpr = {}
tpr = {}
threshold = {}
plt.figure()

tagger = 'ak8_sdmass'
fpr[tagger], tpr[tagger], threshold[tagger] = roc_curve(df['label'], df[tagger], sample_weight=df['sample_weight'])
myauc = auc(fpr[tagger], tpr[tagger],reorder=True)
plt.plot(tpr[tagger],fpr[tagger],label='%s tagger, AUC = %.3f'%(tagger, myauc))

plt.legend(loc='best')
plt.semilogy()
plt.xlim([0, 1])
plt.ylim([1e-4, 1])
plt.xlabel("TPR");
plt.ylabel("FPR");

# checkpoint: 
Compare performance of other tagging algorithms

In [ ]:
## place holder for checkpoint

# Beyond ROC curves: Tagger's response vs. jet mass 
Study the correlation between the tagger's response and the jet mass
Different WP for each algorithm will be provided by the instructor
Discussion: thoughts, ideas, concerns ?

In [ ]:
## place holder for the exercise

# Train your own simple tagger
Train your own simple tagger with keras

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, Activation, Dense

inputs = Input(shape=(2,), name = 'input')  
x = Dense(5, name = 'dense_1', activation = 'relu')(inputs)
outputs = Dense(1, name = 'output', activation='sigmoid')(x)

# creae the model
model = Model(inputs=inputs, outputs=outputs)
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# print the model summary
model.summary()

In [ ]:
x = np.vstack([df['ak8_sdmass'], df['ak8_tau21']]).T
print(x.shape)
y = df['label']
print(y.shape)
model.fit(x,y, epochs=1)
df['ak8_sdmass_tau21'] = model.predict(x)

# Exercises
 - Exercise 1. Plot all of the taggers on the same ROC curve
 - Exercise 2. Train your own custom "traditional" tagger based on mSD and tau21
 - Exercise 3. Add your tagger to the ROC curve
 - Exercise 4. Repeat with a different signal sample (W, Z, or top)
 - Exercise 5 [advanced]. Understand the impact of different sets of inputs in the jet classification problem